# Supervised ANNs for Mutual Information Investigation
## Thomas Possidente

### Imports and Initializations

In [1]:
# ANN Building and Visualization Imports
import keras
from keras import backend as K
from keras import optimizers, losses
from keras.engine.topology import Layer
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
import tensorflow as tf
from keras.callbacks import LambdaCallback
from keras.utils import to_categorical
import matplotlib.pyplot as plt



# Standard Imports
import pandas as pd
import numpy as np
import math

# Value Inits - Specify as needed
num_inputs = int(5000)  # number of dummy images in set
size = int(16)          # Dimension of each dummy image should be size*size
RF_size = int(2)        # Dimensions of the RF to be analyzed should be RF_size*RF_size
noise = 0.5               # percentage (as decimal) of input values that will be flipped 

# Value Inits - Leave these alone
num_of_RFs = int((size*size) / (RF_size*RF_size))


C:\Users\Tom\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Reading in Data

In [2]:
inputs = pd.read_csv('Pattern_Matrices_Datasets/size16_RF8_noise0.csv')
inputs = inputs.drop('X1', axis = 0) # Taking out col names
inputs = inputs.apply(pd.to_numeric)  # converting to floats

inputs = inputs.values # convert to np ndarray
inputs = inputs.reshape(num_inputs, size, size,1) # reshape to desired dims (5000 examples, 16*16 image, 1 channel)
flattened_inputs = inputs.reshape(num_inputs, size*size)

### Cutting Number of Labels Possible to 256 if Necessary

In [108]:
flattened_inputs = flattened_inputs[0:256]
flattened_inputs = np.repeat(flattened_inputs, 20, 0)
flattened_inputs = np.random.permutation(flattened_inputs)

num_inputs = int(flattened_inputs.shape[0])


### Creating Labels

In [3]:
labels = np.zeros(np.shape(flattened_inputs)[0])
storage = np.zeros(np.shape(flattened_inputs))
count = 0

for i in range(np.shape(flattened_inputs)[0]):
    if(~((flattened_inputs[i] == storage).all(1).any())):
        labels[i] = count
        storage[i] = flattened_inputs[i]
        count += 1
    elif((flattened_inputs[i] == storage).all(1).any()):
        label_index = np.where((flattened_inputs[i] == storage).all(1))
        correct_label = labels[label_index]
        storage[i] = flattened_inputs[i]
        labels[i] = correct_label[0]
        
labels = labels.astype(int)
labels = to_categorical(labels)

    

In [4]:
labels.shape

(5000, 256)

### Adding Noise to Inputs

In [5]:
for n in range(np.shape(flattened_inputs)[0]):
    indices_to_flip = np.random.choice(int(size*size), math.ceil(size*size * noise), replace = False)
    flattened_inputs[n][indices_to_flip] = 1 - flattened_inputs[n][indices_to_flip]

inputs = flattened_inputs.reshape(num_inputs, size, size, 1)


In [6]:
model = Sequential()
model.add(Conv2D(input_shape = (16,16,1), filters=4, kernel_size = RF_size, strides = RF_size, activation = 'relu'))
model.add(Flatten())
model.add(Dense(512, activation = 'relu'))
model.add(Dense(256, activation = 'softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 8, 8, 4)           20        
_________________________________________________________________
flatten_1 (Flatten)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               131584    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
Total params: 262,932
Trainable params: 262,932
Non-trainable params: 0
_________________________________________________________________


### Building Network

In [7]:
def run_model():
    
    model = Sequential()
    model.add(Conv2D(input_shape = (16,16,1), filters=4, kernel_size = RF_size, strides = RF_size, activation = 'relu'))
    model.add(Flatten())
    model.add(Dense(512, activation = 'relu'))
    model.add(Dense(256, activation = 'softmax'))
    model.compile(optimizer = optimizers.adam(lr = 0.0005), loss = 'categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(x = inputs, validation_split = 0.25, y = labels, batch_size = 100, epochs = 25, shuffle=False)
    return(history)



In [8]:
storage = np.zeros((6, 8))
param_RF_size = np.array([2,3,4,6,8])
num_inputs = int(5000) 
size = int(16)        

for n in range(5):
    RF_size = int(param_RF_size[n])
    
    val_acc15 = np.zeros(10)
    val_acc99 = np.zeros(10)
    val_loss15 = np.zeros(10)
    val_loss99 = np.zeros(10)
    for i in range(10):
        results = run_model()
        val_acc15[i] = results.history['val_acc'][4]
        val_acc99[i] = results.history['val_acc'][24]
        val_loss15[i] = results.history['val_loss'][4]
        val_loss99[i] = results.history['val_loss'][24]

    acc15 = np.mean(val_acc15)
    acc100 = np.mean(val_acc99)
    loss15 = np.mean(val_loss15)
    loss100 = np.mean(val_loss99)
    acc15_SD = np.std(val_acc15)
    acc100_SD = np.std(val_acc99)
    loss15_SD = np.std(val_loss15)
    loss100_SD = np.std(val_loss99)
    
    one_res = np.array([acc15, acc100, loss15, loss100, acc15_SD, acc100_SD, loss15_SD, loss100_SD])
    
    storage[n] = one_res
    
#print("acc15 = " + str(np.mean(val_acc15)), "\n",
#      "acc100 = " + str(np.mean(val_acc99)), "\n",
#      "loss15 = " + str(np.mean(val_loss15)), "\n",
#      "loss100 = " + str(np.mean(val_loss99)), "\n",
#      "acc15_SD = " + str(np.std(val_acc15)), "\n",
#      "acc100_SD = " + str(np.std(val_acc99)), "\n",
#      "loss15_SD = " + str(np.std(val_loss15)), "\n",
#      "loss100_SD = " + str(np.std(val_loss99)))

Train on 3750 samples, validate on 1250 samples
Epoch 1/25
3750/3750 [==============================] - 1s 248us/step - loss: 5.5792 - acc: 0.0045 - val_loss: 5.5699 - val_acc: 0.0048
Epoch 2/25
3750/3750 [==============================] - 1s 182us/step - loss: 5.4553 - acc: 0.0120 - val_loss: 5.5880 - val_acc: 0.0032
Epoch 3/25
3750/3750 [==============================] - 1s 182us/step - loss: 5.3625 - acc: 0.0235 - val_loss: 5.6090 - val_acc: 0.0016
Epoch 4/25
3750/3750 [==============================] - 1s 191us/step - loss: 5.2359 - acc: 0.0421 - val_loss: 5.6289 - val_acc: 0.0056
Epoch 5/25
3750/3750 [==============================] - 1s 183us/step - loss: 5.0734 - acc: 0.0755 - val_loss: 5.6552 - val_acc: 0.0056
Epoch 6/25
3750/3750 [==============================] - 1s 178us/step - loss: 4.8719 - acc: 0.1269 - val_loss: 5.6923 - val_acc: 0.0048
Epoch 7/25
3750/3750 [==============================] - 1s 166us/step - loss: 4.6371 - acc: 0.1888 - val_loss: 5.7397 - val_acc: 0.0032


Epoch 10/25
3750/3750 [==============================] - 1s 159us/step - loss: 4.0650 - acc: 0.2899 - val_loss: 5.9398 - val_acc: 0.0040
Epoch 11/25
3750/3750 [==============================] - 1s 158us/step - loss: 3.8059 - acc: 0.3333 - val_loss: 6.0588 - val_acc: 0.0040
Epoch 12/25
3750/3750 [==============================] - ETA: 0s - loss: 3.5626 - acc: 0.368 - 1s 167us/step - loss: 3.5493 - acc: 0.3720 - val_loss: 6.1890 - val_acc: 0.0040
Epoch 13/25
3750/3750 [==============================] - 1s 160us/step - loss: 3.3023 - acc: 0.4184 - val_loss: 6.3305 - val_acc: 0.0032
Epoch 14/25
3750/3750 [==============================] - 1s 167us/step - loss: 3.0653 - acc: 0.4589 - val_loss: 6.4780 - val_acc: 0.0032
Epoch 15/25
3750/3750 [==============================] - 1s 159us/step - loss: 2.8389 - acc: 0.4952 - val_loss: 6.6317 - val_acc: 0.0032
Epoch 16/25
3750/3750 [==============================] - 1s 161us/step - loss: 2.6246 - acc: 0.5368 - val_loss: 6.7880 - val_acc: 0.0032
Epo

Epoch 19/25
3750/3750 [==============================] - 1s 156us/step - loss: 1.4849 - acc: 0.7811 - val_loss: 7.4589 - val_acc: 0.0024
Epoch 20/25
3750/3750 [==============================] - 1s 155us/step - loss: 1.3152 - acc: 0.8171 - val_loss: 7.6511 - val_acc: 0.0032
Epoch 21/25
3750/3750 [==============================] - 1s 168us/step - loss: 1.1619 - acc: 0.8435 - val_loss: 7.8417 - val_acc: 0.0024
Epoch 22/25
3750/3750 [==============================] - 1s 156us/step - loss: 1.0238 - acc: 0.8720 - val_loss: 8.0331 - val_acc: 0.0032
Epoch 23/25
3750/3750 [==============================] - 1s 157us/step - loss: 0.8988 - acc: 0.9027 - val_loss: 8.2285 - val_acc: 0.0040
Epoch 24/25
3750/3750 [==============================] - 1s 169us/step - loss: 0.7871 - acc: 0.9232 - val_loss: 8.4208 - val_acc: 0.0040
Epoch 25/25
3750/3750 [==============================] - 1s 159us/step - loss: 0.6884 - acc: 0.9395 - val_loss: 8.6127 - val_acc: 0.0040
Train on 3750 samples, validate on 1250 s

Epoch 3/25
3750/3750 [==============================] - 1s 195us/step - loss: 5.4377 - acc: 0.0275 - val_loss: 5.5802 - val_acc: 0.0024
Epoch 4/25
3750/3750 [==============================] - 1s 198us/step - loss: 5.3524 - acc: 0.0245 - val_loss: 5.6061 - val_acc: 0.0032
Epoch 5/25
3750/3750 [==============================] - 1s 179us/step - loss: 5.2375 - acc: 0.0451 - val_loss: 5.6218 - val_acc: 0.0040
Epoch 6/25
3750/3750 [==============================] - 1s 195us/step - loss: 5.0870 - acc: 0.0872 - val_loss: 5.6366 - val_acc: 0.0016
Epoch 7/25
3750/3750 [==============================] - 1s 194us/step - loss: 4.9027 - acc: 0.1440 - val_loss: 5.6506 - val_acc: 0.0016
Epoch 8/25
3750/3750 [==============================] - 1s 189us/step - loss: 4.6887 - acc: 0.2157 - val_loss: 5.6793 - val_acc: 0.0032
Epoch 9/25
3750/3750 [==============================] - 1s 179us/step - loss: 4.4480 - acc: 0.2837 - val_loss: 5.7265 - val_acc: 0.0024
Epoch 10/25
3750/3750 [=========================

3750/3750 [==============================] - 1s 197us/step - loss: 3.3477 - acc: 0.4235 - val_loss: 6.1964 - val_acc: 0.0064
Epoch 13/25
3750/3750 [==============================] - 1s 201us/step - loss: 3.0602 - acc: 0.4755 - val_loss: 6.3555 - val_acc: 0.0064
Epoch 14/25
3750/3750 [==============================] - 1s 186us/step - loss: 2.7866 - acc: 0.5200 - val_loss: 6.5259 - val_acc: 0.0072
Epoch 15/25
3750/3750 [==============================] - 1s 197us/step - loss: 2.5280 - acc: 0.5675 - val_loss: 6.7015 - val_acc: 0.0080
Epoch 16/25
3750/3750 [==============================] - 1s 195us/step - loss: 2.2864 - acc: 0.6083 - val_loss: 6.8813 - val_acc: 0.0088
Epoch 17/25
3750/3750 [==============================] - 1s 197us/step - loss: 2.0620 - acc: 0.6445 - val_loss: 7.0668 - val_acc: 0.0072
Epoch 18/25
3750/3750 [==============================] - 1s 190us/step - loss: 1.8542 - acc: 0.6813 - val_loss: 7.2555 - val_acc: 0.0072
Epoch 19/25
3750/3750 [==============================

...

Epoch 4/25
3750/3750 [==============================] - 1s 153us/step - loss: 5.4603 - acc: 0.0104 - val_loss: 5.5886 - val_acc: 0.0016
Epoch 5/25
3750/3750 [==============================] - 1s 144us/step - loss: 5.4258 - acc: 0.0115 - val_loss: 5.6022 - val_acc: 0.0016
Epoch 6/25
3750/3750 [==============================] - 1s 146us/step - loss: 5.3857 - acc: 0.0139 - val_loss: 5.6185 - val_acc: 0.0032
Epoch 7/25
3750/3750 [==============================] - 1s 157us/step - loss: 5.3409 - acc: 0.0157 - val_loss: 5.6368 - val_acc: 0.0024
Epoch 8/25
3750/3750 [==============================] - 1s 143us/step - loss: 5.2926 - acc: 0.0200 - val_loss: 5.6575 - val_acc: 0.0024
Epoch 9/25
3750/3750 [==============================] - 1s 147us/step - loss: 5.2427 - acc: 0.0232 - val_loss: 5.6838 - val_acc: 0.0024
Epoch 10/25
3750/3750 [==============================] - 1s 153us/step - loss: 5.1932 - acc: 0.0304 - val_loss: 5.7161 - val_acc: 0.0024
Epoch 11/25
3750/3750 [========================

3750/3750 [==============================] - 1s 158us/step - loss: 5.0113 - acc: 0.0437 - val_loss: 5.8294 - val_acc: 0.0048
Epoch 14/25
3750/3750 [==============================] - 1s 153us/step - loss: 4.9718 - acc: 0.0448 - val_loss: 5.8720 - val_acc: 0.0048
Epoch 15/25
3750/3750 [==============================] - 1s 147us/step - loss: 4.9351 - acc: 0.0469 - val_loss: 5.9134 - val_acc: 0.0040
Epoch 16/25
3750/3750 [==============================] - 1s 161us/step - loss: 4.9011 - acc: 0.0469 - val_loss: 5.9541 - val_acc: 0.0040
Epoch 17/25
3750/3750 [==============================] - 1s 149us/step - loss: 4.8694 - acc: 0.0491 - val_loss: 5.9926 - val_acc: 0.0040
Epoch 18/25
3750/3750 [==============================] - 1s 150us/step - loss: 4.8393 - acc: 0.0520 - val_loss: 6.0308 - val_acc: 0.0040
Epoch 19/25
3750/3750 [==============================] - 1s 160us/step - loss: 4.8111 - acc: 0.0539 - val_loss: 6.0671 - val_acc: 0.0048
Epoch 20/25
3750/3750 [==============================

3750/3750 [==============================] - 1s 166us/step - loss: 4.7476 - acc: 0.0600 - val_loss: 6.1254 - val_acc: 8.0000e-04
Epoch 23/25
3750/3750 [==============================] - 1s 154us/step - loss: 4.7202 - acc: 0.0640 - val_loss: 6.1531 - val_acc: 8.0000e-04
Epoch 24/25
3750/3750 [==============================] - 1s 153us/step - loss: 4.6928 - acc: 0.0677 - val_loss: 6.1821 - val_acc: 8.0000e-04
Epoch 25/25
3750/3750 [==============================] - 1s 194us/step - loss: 4.6655 - acc: 0.0723 - val_loss: 6.2095 - val_acc: 8.0000e-04
Train on 3750 samples, validate on 1250 samples
Epoch 1/25
3750/3750 [==============================] - 2s 581us/step - loss: 5.5473 - acc: 0.0029 - val_loss: 5.5475 - val_acc: 0.0024
Epoch 2/25
3750/3750 [==============================] - 1s 154us/step - loss: 5.5206 - acc: 0.0072 - val_loss: 5.5546 - val_acc: 0.0040
Epoch 3/25
3750/3750 [==============================] - 1s 154us/step - loss: 5.4903 - acc: 0.0088 - val_loss: 5.5696 - val_acc:

Epoch 6/25
3750/3750 [==============================] - 1s 165us/step - loss: 5.3691 - acc: 0.0168 - val_loss: 5.6152 - val_acc: 0.0024
Epoch 7/25
3750/3750 [==============================] - 1s 155us/step - loss: 5.3233 - acc: 0.0224 - val_loss: 5.6299 - val_acc: 0.0024
Epoch 8/25
3750/3750 [==============================] - 1s 166us/step - loss: 5.2745 - acc: 0.0237 - val_loss: 5.6494 - val_acc: 0.0024
Epoch 9/25
3750/3750 [==============================] - 1s 155us/step - loss: 5.2245 - acc: 0.0251 - val_loss: 5.6742 - val_acc: 0.0016
Epoch 10/25
3750/3750 [==============================] - 1s 157us/step - loss: 5.1750 - acc: 0.0275 - val_loss: 5.7046 - val_acc: 0.0016
Epoch 11/25
3750/3750 [==============================] - 1s 168us/step - loss: 5.1274 - acc: 0.0283 - val_loss: 5.7395 - val_acc: 0.0016
Epoch 12/25
3750/3750 [==============================] - 1s 157us/step - loss: 5.0829 - acc: 0.0344 - val_loss: 5.7769 - val_acc: 0.0016
Epoch 13/25
3750/3750 [======================

3750/3750 [==============================] - 1s 161us/step - loss: 4.9490 - acc: 0.0565 - val_loss: 5.9213 - val_acc: 0.0056
Epoch 16/25
3750/3750 [==============================] - 1s 170us/step - loss: 4.9150 - acc: 0.0587 - val_loss: 5.9610 - val_acc: 0.0056
Epoch 17/25
3750/3750 [==============================] - 1s 159us/step - loss: 4.8833 - acc: 0.0589 - val_loss: 5.9993 - val_acc: 0.0048
Epoch 18/25
3750/3750 [==============================] - 1s 159us/step - loss: 4.8531 - acc: 0.0608 - val_loss: 6.0354 - val_acc: 0.0048
Epoch 19/25
3750/3750 [==============================] - 1s 171us/step - loss: 4.8244 - acc: 0.0629 - val_loss: 6.0703 - val_acc: 0.0048
Epoch 20/25
3750/3750 [==============================] - 1s 160us/step - loss: 4.7966 - acc: 0.0661 - val_loss: 6.1036 - val_acc: 0.0048
Epoch 21/25
3750/3750 [==============================] - 1s 170us/step - loss: 4.7695 - acc: 0.0683 - val_loss: 6.1356 - val_acc: 0.0048
Epoch 22/25
3750/3750 [==============================

In [9]:
storage

array([[4.55999990e-03, 4.39999990e-03, 5.63316564e+00, 8.48933345e+00,
        1.38794809e-03, 2.12414684e-03, 9.47671172e-03, 1.45830508e-01],
       [2.87999994e-03, 3.51999992e-03, 5.62125978e+00, 7.64534548e+00,
        1.02449986e-03, 1.56767340e-03, 1.04571201e-02, 1.03240424e-01],
       [3.51999992e-03, 3.27999993e-03, 5.61607899e+00, 7.25059834e+00,
        1.24963992e-03, 1.21061965e-03, 6.95202617e-03, 6.17776430e-02],
       [3.51999992e-03, 3.99999991e-03, 5.60019398e+00, 6.18217104e+00,
        1.60798006e-03, 2.29085132e-03, 4.11125980e-03, 3.87551421e-02],
       [3.35999992e-03, 2.95999993e-03, 5.59991209e+00, 6.22076390e+00,
        1.11999997e-03, 1.24193395e-03, 2.64364437e-03, 2.46607574e-02],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [10]:
res = storage

In [ ]:
storage = np.zeros((60, 11))
param_RF_size = np.array([2,3,4,6,8])
param_noise = np.array([0, 0.1, 0.2, 0.3, 0.4, 0.5])
param_file = np.array(["Pattern_Matrices_Datasets/size16_RF2_noise0.csv", "Pattern_Matrices_Datasets/size16_RF8_noise0.csv"])
num_inputs = int(5000) 
size = int(16)        

counter99 = 0

for f in range(2):
    file = param_file[f]
    
    for n in range(6):
        noise = param_noise[n]
        
        # Read in #
        inputs = pd.read_csv(file)
        inputs = inputs.drop('X1', axis = 0) # Taking out col names
        inputs = inputs.apply(pd.to_numeric)  # converting to floats

        inputs = inputs.values # convert to np ndarray
        inputs = inputs.reshape(num_inputs, size, size,1) # reshape to desired dims (5000 examples, 16*16 image, 1 channel)
        flattened_inputs = inputs.reshape(num_inputs, size*size)

        # Cutting Labels #
        flattened_inputs = flattened_inputs[0:256]
        flattened_inputs = np.repeat(flattened_inputs, 20, 0)
        flattened_inputs = np.random.permutation(flattened_inputs)
        num_inputs = int(flattened_inputs.shape[0])

        # Create Labels #
        labels = np.zeros(np.shape(flattened_inputs)[0])
        storage = np.zeros(np.shape(flattened_inputs))
        count = 0
        for a in range(np.shape(flattened_inputs)[0]):
            if(~((flattened_inputs[a] == storage).all(1).any())):
                labels[a] = count
                storage[a] = flattened_inputs[a]
                count += 1
            elif((flattened_inputs[a] == storage).all(1).any()):
                label_index = np.where((flattened_inputs[a] == storage).all(1))
                correct_label = labels[label_index]
                storage[a] = flattened_inputs[i]
                labels[a] = correct_label[0]

        labels = labels.astype(int)
        labels = to_categorical(labels)

        # Add Noise #
        for s in range(np.shape(flattened_inputs)[0]):
            indices_to_flip = np.random.choice(int(size*size), math.ceil(size*size * noise), replace = False)
            flattened_inputs[s][indices_to_flip] = 1 - flattened_inputs[s][indices_to_flip]

        inputs = flattened_inputs.reshape(num_inputs, size, size, 1)


        # Build Model #
        def run_model():
            model = Sequential()
            model.add(Conv2D(input_shape = (16,16,1), filters=4, kernel_size = RF_size, strides = RF_size, activation = 'relu'))
            model.add(Flatten())
            model.add(Dense(512, activation = 'relu'))
            model.add(Dense(256, activation = 'softmax'))
            model.compile(optimizer = optimizers.adam(lr = 0.0005), loss = 'categorical_crossentropy', metrics=['accuracy'])
            history = model.fit(x = inputs, validation_split = 0.25, y = labels, batch_size = 100, epochs = 25, shuffle=False)
            return(history)
    
        for r in range(5):
            RF_size = int(param_RF_size[n])

            val_acc15 = np.zeros(10)
            val_acc99 = np.zeros(10)
            val_loss15 = np.zeros(10)
            val_loss99 = np.zeros(10)

            
            for i in range(10):
                results = run_model()
                val_acc15[i] = results.history['val_acc'][4]
                val_acc99[i] = results.history['val_acc'][24]
                val_loss15[i] = results.history['val_loss'][4]
                val_loss99[i] = results.history['val_loss'][24]

            acc15 = np.mean(val_acc15)
            acc100 = np.mean(val_acc99)
            loss15 = np.mean(val_loss15)
            loss100 = np.mean(val_loss99)
            acc15_SD = np.std(val_acc15)
            acc100_SD = np.std(val_acc99)
            loss15_SD = np.std(val_loss15)
            loss100_SD = np.std(val_loss99)

            one_res = np.array([file, noise, RF_size, acc15, acc100, loss15, loss100, acc15_SD, acc100_SD, loss15_SD, loss100_SD])

            storage[counter99] = one_res
            counter99 += 1



In [195]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

NameError: name 'history' is not defined

### TODOS
* Find Image dataset that's binarized (or binarize existing one) and do MI calculations to find max MI. To do this, sample RF sized chunks of an image until stable average MI value is reached
* https://people.cs.umass.edu/~marlin/data.shtml, http://vision.lems.brown.edu/content/available-software-and-databases